In [7]:
from tfim_sk_infd.services import jij_service
from tfim_sk_infd.models.SKSpinGlass import SKSpinGlass
import numpy
import sys

numpy.set_printoptions(threshold=sys.maxsize)

T = 4

Jij_instance = jij_service.get_toy_Jij(T)
SG = SKSpinGlass(Jij_instance)

print(SG.JZZ.get_shape())

print(SG.JZZ.diagonal() )

100% |########################################################################|


Building JZZ...
Done.
Building Mx...
Done.
(16, 16)
[-2.  2.  2.  2.  0.  0.  0. -4. -4.  0.  0.  0.  2.  2.  2. -2.]


In [4]:
import numpy as np

# Original NumPy array
original_array = np.array([[1, 2, 3], [4, 5, 6]])

# Convert the NumPy array to a string
array_string = np.array2string(original_array)

In [5]:
print(array_string)

[[1 2 3]
 [4 5 6]]


In [9]:
import json
original_array = np.array([[1, 2, 3], [4, 5, 6]])

arr_str = json.dumps(original_array.tolist())
np.array(json.loads(arr_str))

array([[1, 2, 3],
       [4, 5, 6]])

In [10]:
N=8
(N*(N-1)) / 2

28.0

In [47]:
from tfim_sk_infd.services import jij_service, ground_state_service, qutip_service
import numpy as np

# Get an instance and find ground state
seed = 0
N = 8
Jij = jij_service.Jij_instance(N, seed)
JZZ = jij_service.JZZ_SK(Jij)
gs_list = np.where(JZZ.diagonal() == JZZ.diagonal().max())[0]

# Pull out half of the ground states and determine maximum hd
reduced_gs, max_h_d = ground_state_service.maximal_half_clique(gs_list, N)

print(reduced_gs)

gs_size = len(reduced_gs)

# Find quantum annealing final state
Hp = qutip_service.spin_glass_hamiltonian(Jij, N)
Ht = qutip_service.transverse_field_hamiltonian(N)

h = 1e-2
h_t = (-Hp*(1-h)) - (h * Ht)

eigenvalues, eigenstates = h_t.eigenstates(eigvals = 1)

# print(eigenstates[0])

annealing_gs_probs = qutip_service.quantum_anneal(Hp, Ht, N, 1000, 100)

[9, 30]


In [48]:
eigenstates[0].full().flatten()[gs_list]

array([0.49998355+0.j, 0.499984  +0.j, 0.49998385+0.j, 0.49998421+0.j])

In [34]:
np.array(annealing_gs_probs)[gs_list]

array([0.24999996, 0.24999996, 0.24999996, 0.24999996])

In [51]:
for h in (1, 0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5 ):
    h_t = (-Hp*(1-h)) - (h * Ht)

    # Calculate condition number
    cond_number = np.linalg.cond(h_t.full())
    print("Condition number:", cond_number)

print(np.linalg.cond(Hp.full()))

Condition number: 8.621715046583702e+17
Condition number: inf
Condition number: 1.8073537748923588e+16
Condition number: 1.1143847909944384e+17
Condition number: 6.501768394827755e+16
Condition number: 1.2890019807250006e+17
Condition number: 1.2631674856366051e+17
inf


In [60]:
h_test = Hp.full()[1:6,1:6]
print(h_test)
print(np.linalg.cond(h_test))

[[ 6.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -6.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -4.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  2.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  4.+0.j]]
3.0


In [20]:
from tfim_sk_infd.services import ground_state_service
import numpy as np

N = 8
gs = [9,30]

overlap_dist = ground_state_service.overlap_distribution_fair(gs, N)
overlap_dist = overlap_dist.tolist()
od_mean = np.mean(overlap_dist)
manual_mean = sum([ ( ((2*i)/N) - 1) * overlap_dist[i] for i in range(N+1)])
manual_mean_bucket = sum([ i * overlap_dist[i] for i in range(N+1)])
od_variance = np.var(overlap_dist)
manual_variance = sum([ (( (((2*i)/N) - 1) **2) * overlap_dist[i]) - (manual_mean**2) for i in range(N+1)])
manual_variance_bucket = sum([ ((i**2) * overlap_dist[i]) - (manual_mean_bucket**2) for i in range(N+1)])

hd = ground_state_service.base_N_hamming_distance(9,30,8)

In [21]:

bucket_vals = [ ( ((2*i)/N) - 1)  for i in range(N+1)]

print(overlap_dist)
print(manual_mean)
print(manual_mean_bucket)
print(bucket_vals[int(manual_mean_bucket)])
print(od_mean)
print(od_variance)
print(manual_variance)
print(manual_variance_bucket)
print(hd)

[0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5]
0.5
6.0
0.5
0.1111111111111111
0.043209876543209874
-1.75
-284.0
4


In [22]:
N = 12
[((2*i)/N) - 1 for i in range(N+1)]
np.var(np.array([0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5]))

0.043209876543209874

In [14]:
import data_service
from database.models import InstancesN8
from tfim_sk_infd.services import qutip_service
from tfim_sk_infd.models.Jij import Jij
import numpy as np

N = 8
h = 1e-2

with data_service.get_session() as session:

    instance = session.query(InstancesN8).where(InstancesN8.seed == 1).all()[0]
    Jij_obj = Jij(np.array(instance.Jij_matrix))

    Ht = qutip_service.transverse_field_hamiltonian(N)
    Hp = qutip_service.spin_glass_hamiltonian(Jij_obj, N)

    h_t = -Hp - h * Ht

    eigenvalues, eigenstates = h_t.eigenstates(eigvals=1)

    gs_probs = [
        round(2 * float(pa.real**2 + pa.imag**2),5)
        for pa in eigenstates[0].full().flatten()[instance.reduced_gs]
    ]

    if abs(sum(gs_probs) - 1) > 1e-2:
        print(
            f"seed {instance.seed} diagonalization fail. gs prob array sum {sum(gs_probs)}\n{gs_probs}"
        )

In [15]:
gs_probs

[0.49997, 0.49997]

In [3]:
from tqdm import tqdm
import time

# Example loop
for i in tqdm(range(100)):
    # Simulate some work with a sleep
    time.sleep(0.1)

100%|██████████| 100/100 [00:10<00:00,  9.49it/s]


In [1]:
"""
python3 -m scripts.add_instances           
python3 -m scripts.update_degeneracy_col
python3 -m scripts.update_reduced_gs_hd
python3 -m scripts.update_overlap_dist 
python3 -m scripts.update_post_anneal_gs_probs
python3 -m scripts.update_post_anneal_cols    
python3 -m scripts.update_post_anneal_gs_hd
"""

'\npython3 -m scripts.add_instances           \npython3 -m scripts.update_degeneracy_col\npython3 -m scripts.update_reduced_gs_hd\npython3 -m scripts.update_overlap_dist \npython3 -m scripts.update_post_anneal_gs_probs\npython3 -m scripts.update_post_anneal_cols    \npython3 -m scripts.update_post_anneal_gs_hd\n'

In [28]:
import numpy as np
from scipy import linalg

from database.models import InstancesN12
from tfim_sk_infd.services import qutip_service
from tfim_sk_infd.models.Jij import Jij
from tfim_sk_infd.models.SKSpinGlass import SKSpinGlass
import data_service

N = 12
h = 1e-1


with data_service.get_session() as session:

    instances = (
        session.query(InstancesN12)
        .where(InstancesN12.degeneracy > 2)
        .all()
    )

    Ht = qutip_service.transverse_field_hamiltonian(N)
    for instance in instances[:2]:


        Jij_obj = Jij(np.array(instance.Jij_matrix))

        # QuTiP
        Hp = qutip_service.spin_glass_hamiltonian(Jij_obj, N)
        h_t = -Hp - h * Ht
        eigenvalues, eigenstates = h_t.eigenstates(eigvals=1)
        gs_probs_q = [
            round(2 * float(pa.real**2 + pa.imag**2), 5)
            for pa in eigenstates[0].full().flatten()[instance.reduced_gs]
        ]
        gs_probs_q_full = [
            round(float(pa.real**2 + pa.imag**2), 5)
            for pa in eigenstates[0].full().flatten()[instance.ground_states]
        ]

        

        # tfim

        SG = SKSpinGlass(Jij_obj)

        # Define Hamiltonian H at h
        H = SG.hamiltonian_at_h(h)

        # Diagonalize H and get ground state vector psi0
        E, v = linalg.eigh(H.todense())
        sort_order = np.argsort(E)
        v = v[:, sort_order]
        psi0 = v[:, 0]

        # retrieve and format probabilities of classical ground states in psi0
        gs_probs_tfim = np.array([2* round(psi0[g] ** 2 ,5)for g in instance.reduced_gs])
        gs_probs_tfim_full = np.array([round(psi0[g] ** 2,5) for g in instance.ground_states])


        # print gs probs

        print("-------------------")
        print(gs_probs_q)
        print(gs_probs_q_full)
        print(gs_probs_tfim)
        print(gs_probs_tfim_full)
        print("-------------------")



    session.commit()

    session.close()


Building JZZ...


100% |########################################################################|


Done.
Building Mx...


100% |########################################################################|


Done.
-------------------
[6e-05, 0.98599]
[3e-05, 0.49299, 0.49822, 3e-05]
[6.0000e-05 9.9432e-01]
[3.0000e-05 4.9716e-01 4.9406e-01 3.0000e-05]
-------------------


Building JZZ...


100% |########################################################################|


Done.
Building Mx...


100% |########################################################################|


Done.
-------------------
[0.21411, 0.21977, 0.05863, 0.21597, 0.0, 0.21278, 0.05013]
[0.10706, 0.10988, 0.0, 0.02931, 0.10639, 0.10799, 0.02506, 0.02506, 0.10799, 0.10639, 0.02931, 0.0, 0.10988, 0.10706]
[0.21412 0.21976 0.05862 0.21598 0.      0.21278 0.05012]
[0.10706 0.10988 0.      0.02931 0.10639 0.10799 0.02506 0.02506 0.10799
 0.10639 0.02931 0.      0.10988 0.10706]
-------------------


In [10]:
o_list = [1.0, 0.6666666666666666, 1.0, 0.6666666666666666, 1.0, 0.3333333333333333, 1.0, 0.3333333333333333, 1.0, 0.0, 1.0, 0.3333333333333333, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [16]:
import numpy as np



np.histogram(o_list, bins = 13, range=(-1.0,1.0))

(array([ 0,  0,  0,  0,  0,  0, 16,  0,  3,  0,  2,  0,  7]),
 array([-1.        , -0.84615385, -0.69230769, -0.53846154, -0.38461538,
        -0.23076923, -0.07692308,  0.07692308,  0.23076923,  0.38461538,
         0.53846154,  0.69230769,  0.84615385,  1.        ]))

In [24]:
def base_N_hamming_distance(i: int, j: int, N: int) -> int:
    arr_i = np.array([int(s) for s in bin(i)[2:].zfill(N)])
    arr_j = np.array([int(s) for s in bin(j)[2:].zfill(N)])
    return int(np.sum(arr_i != arr_j))

gs_list = [395, 451, 1219, 2507, 2963, 1483, 2441]
gs_probs = [0.21411, 0.21977, 0.05863, 0.21597, 0.0, 0.21278, 0.05013]
hd_histogram = np.zeros(13)
gs_size = len(gs_list)
for i in range(gs_size):
    for j in range(i, gs_size):
        h_d = base_N_hamming_distance(gs_list[i], gs_list[j], N)
        if i != j:
            hd_histogram[h_d] += 2* gs_probs[i] * gs_probs[j]
        else:
            hd_histogram[h_d]+= gs_probs[i] * gs_probs[j]

In [25]:
hd_histogram

array([0.19201081, 0.        , 0.65191084, 0.        , 0.09379864,
       0.        , 0.00587824, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        ])

In [26]:
hd_histogram.sum()

0.9435985321

In [2]:
from tfim_sk_infd.services.ground_state_service import overlap_distribution_fair, overlap_distribution_weighted

N = 8
gs_list = [395, 451, 1219, 2507, 2963, 1483, 2441]
gs_probs = [0.21411, 0.21977, 0.05863, 0.21597, 0.0, 0.21278, 0.05013]

print(overlap_distribution_fair(gs_list,N))
print(sum(overlap_distribution_fair(gs_list,N)))
print(overlap_distribution_weighted(gs_list,gs_probs, N))
print(sum(overlap_distribution_weighted(gs_list,gs_probs, N)))


ValueError: operands could not be broadcast together with shapes (9,) (11,) 

In [30]:
from tfim_sk_infd.services import ground_state_service
import numpy as np

N = 8
gs = [9,30]

bins = np.array([((2*i)/N) - 1 for i in range(N+1)])
overlap_dist = ground_state_service.overlap_distribution_fair(gs, N)

mean = np.sum(bins*overlap_dist)
variance = np.sum(overlap_dist * (bins - mean) ** 2)

print(mean)
print(variance)

0.5
0.25


In [31]:
from tfim_sk_infd.services import ground_state_service
N = 12
gs_list = [212, 340, 470]

h_d_dict = {}

for i in range(len(gs_list)):
    for j in range(i+1, len(gs_list)):
        x = gs_list[i]
        y = gs_list[j]
        h_d_dict[f"{x}_{y}"] = ground_state_service.base_N_hamming_distance(x,y,N)

h_d_dict



{'212_340': 2, '212_470': 2, '340_470': 2}

In [68]:
import numpy as np

from database.models import InstancesN8, InstancesN12
from tfim_sk_infd.services import qutip_service
from tfim_sk_infd.models.Jij import Jij
from tfim_sk_infd.models.SKSpinGlass import SKSpinGlass
import data_service
from tqdm import tqdm
from scipy import linalg
from scipy.sparse import linalg as spla

N = 8


Instance = data_service.get_instance_class(N)


with data_service.get_session() as session:

    instances: list[InstancesN8 | InstancesN12] = (
        session.query(Instance).where(Instance.degeneracy > 2).limit(10).all()
    )

    for instance in instances:

        Jij_obj = Jij(np.array(instance.Jij_matrix))

        SG = SKSpinGlass(Jij_obj)

        # initial diag
        h = 0.1
        H = SG.hamiltonian_at_h(h)
        E, v = spla.eigsh(H, k=instance.degeneracy + 1, which="SA")

        sort_order = np.argsort(E)
        E = E[sort_order]
        v = v[:, sort_order]
        psi0 = v[:, 0]

        min_e_gap = E[1] - E[0]
        if min_e_gap < 1e-15:
            print(f"Diagonalization fail for seed {instance.seed} with min_e_gap {min_e_gap} at h = {h}")
            continue

        fidelity = 0
        while(fidelity < 0.99):
            print("------")
            print(h)
            print(2*psi0[instance.reduced_gs]**2)
            print(2*sum(psi0[instance.reduced_gs]**2))
            h /= 2
            H = SG.hamiltonian_at_h(h)
            E, v = spla.eigsh(H, k=instance.degeneracy + 1, which="SA", v0 = psi0)

            sort_order = np.argsort(E)
            E = E[sort_order]
            v = v[:, sort_order]

            old_psi0 = psi0
            psi0 = v[:, 0]
            
            min_e_gap = E[1] - E[0]
            if min_e_gap < 1e-15:
                print(f"Diagonalization fail for seed {instance.seed} with min_e_gap {min_e_gap} at h = {h}")
                break


            fidelity = np.abs(np.inner(psi0,old_psi0))
            
            


        print("------")
        print(h)
        print(2*psi0[instance.reduced_gs]**2)
        print(2*sum(psi0[instance.reduced_gs]**2))
        print("+++++++++++++++++++++++++")

    session.commit()

    session.close()


------
0.1
[0.49683465 0.49683465]
0.9936692901055844
------
0.05
[0.49921049 0.49921049]
0.9984209716376728
+++++++++++++++++++++++++
------
0.1
[0.45583859 0.53783059]
0.9936691768708635
------
0.05
[0.45804474 0.54037622]
0.9984209555396175
+++++++++++++++++++++++++
------
0.1
[0.1162232  0.20403991 0.3348821  0.1162232  0.20403991]
0.97540830857282
------
0.05
[0.11783242 0.2079024  0.34209439 0.11783242 0.2079024 ]
0.9935640225519484
+++++++++++++++++++++++++
------
0.1
[0.22032272 0.54336332 0.22032272]
0.9840087712887097
------
0.05
[0.22247256 0.55094625 0.22247256]
0.9958913677160331
+++++++++++++++++++++++++
------
0.1
[0.22032272 0.54336332 0.22032272]
0.9840087716621786
------
0.05
[0.22247256 0.55094625 0.22247256]
0.995891365957721
+++++++++++++++++++++++++
------
0.1
[0.49564181 0.49564181]
0.99128362813417
------
0.05
[0.49891093 0.49891093]
0.9978218538978142
+++++++++++++++++++++++++
------
0.1
[0.49350794 0.49350794]
0.9870158750136575
------
0.05
[0.49834266 0.49834

In [56]:
abs(np.inner(psi0,old_psi0))

0.9970200238510123

In [44]:
a = 1
a /= 2
a

0.5

In [109]:
import numpy as np

from database.models import InstancesN8, InstancesN12
from tfim_sk_infd.models.Jij import Jij
from tfim_sk_infd.models.SKSpinGlass import SKSpinGlass
import data_service
from scipy.sparse import linalg as spla

N = 12


Instance = data_service.get_instance_class(N)


with data_service.get_session() as session:

    instances: list[InstancesN8 | InstancesN12] = (
        session.query(Instance).where(Instance.degeneracy > 2).where(Instance.seed == 26).all()
    )
    # hx = []
    # min_e_list = []
    # fid_list = []
    for instance in instances:

        try:

            Jij_obj = Jij(np.array(instance.Jij_matrix))

            SG = SKSpinGlass(Jij_obj)

            # initial diag
            h = 0.4
            H = SG.hamiltonian_at_h(h)
            E, v = spla.eigsh(H, k=2, which="SA")

            sort_order = np.argsort(E)
            E = E[sort_order]
            v = v[:, sort_order]
            psi0 = v[:, 0]

            min_e_gap = E[1] - E[0]
            fidelity = 0
            if min_e_gap < 1e-12:
                raise Exception(
                    f"Diagonalization fail for seed {instance.seed} with min_e_gap {min_e_gap} at h = {h}"
                )
            
            print(h)
            
            # hx.append(h)
            # min_e_list.append(min_e_gap)
            
            # fid_list.append(fidelity)
            while fidelity < 0.99999:
                h /= 1.5
                print(h)
                H = SG.hamiltonian_at_h(h)
                # E, v = spla.eigsh(H, k=2, which="SA", v0=psi0)
                E, v = spla.eigsh(H, k=2, which="SA")

                sort_order = np.argsort(E)
                E = E[sort_order]
                v = v[:, sort_order]

                old_psi0 = psi0
                psi0 = v[:, 0]

                fidelity = np.abs(np.vdot(psi0, old_psi0))

                min_e_gap = E[1] - E[0]
                if min_e_gap < 1e-12:
                    raise Exception(
                        f"Diagonalization fail for seed {instance.seed} with min_e_gap {min_e_gap} at h = {h}"
                    )

                

            gs_probs = psi0[instance.ground_states]
            print(gs_probs)
            print(h)
            print(min_e_gap)
            print(fidelity)

        except Exception as e:
            print(
                f"Diagonalization fail for seed {instance.seed}, min_e_gap {min_e_gap}, h {h}, fidelity {fidelity}\norgiginal error:\n{e}"
            )

    session.close()

0.4
0.26666666666666666
0.17777777777777778
0.11851851851851852
0.07901234567901234
Diagonalization fail for seed 26, min_e_gap 1.0658141036401503e-13, h 0.07901234567901234, fidelity 0.8936763717000806
orgiginal error:
Diagonalization fail for seed 26 with min_e_gap 1.0658141036401503e-13 at h = 0.07901234567901234


In [111]:
# Graph min e gap vs hx
classical_gs = [212, 340, 470, 3625, 3755, 3883]
print(old_psi0[classical_gs])
print(np.sum(old_psi0[classical_gs]**2))

[-0.36348421 -0.36925948 -0.46697691 -0.46687762 -0.36918098 -0.36340693]
0.9728746725424222
